In [2]:
import openai, os

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

deployment_name = "text-davinci-003"

# Completion 接口应用

Completion 接口用于内容补全

In [2]:
prompt = ["问题：介绍一下墨问西东是什么类型的公司\n回答："]
response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, temperature=0, max_tokens=1024)
print(response['choices'][0]['text']) 

墨问西东是一家专注于智能家居技术研发的高科技公司，主要从事智能家居产品的研发、生产、销售和服务。公司拥有一支技术精湛、经验丰富的研发团队，致力于为客户提供最先进的智能家居解决方案。


In [14]:
engine = "text-davinci-003"
# engine = "gpt-35-turbo"

prompt = ["问题：介绍一下墨问西东是什么类型的公司"]
response = openai.Completion.create(engine=engine, prompt=prompt, temperature=0, max_tokens=1024)
# print(response)
print(response["choices"][0]["text"])



墨问西东是一家专注于智能家居、智能安防、智能家电、智能家庭网络等智能科技领域的高科技企业。公司致力于为客户提供全方位的智能家居解决方案，以满足客户的智能家居需求。公司拥有一支专业的技术团队，以及一套完善的研发体系，为客户提供最先进的智能家居解决方案。


In [12]:
prompt = ["Can you tell me who are you"]

response = openai.Completion.create(engine=engine, prompt=prompt, temperature=0.1, max_tokens=1024)
print(response["choices"][0]["text"])



I am a person who is interested in learning new things and exploring the world. I am passionate about helping others and making a positive impact in the world.


In [13]:
response

<OpenAIObject text_completion id=cmpl-82L3c4nEuwAUBxpLTpfaxwTZKO2QZ at 0x11881de00> JSON: {
  "id": "cmpl-82L3c4nEuwAUBxpLTpfaxwTZKO2QZ",
  "object": "text_completion",
  "created": 1695568292,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nI am a person who is interested in learning new things and exploring the world. I am passionate about helping others and making a positive impact in the world.",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 7,
    "total_tokens": 40
  }
}

# ChatCompletion接口应用

ChatCompletion 接口对聊天场景做了优化，统一规范了格式，提供了`system`, `assistent` 和 `user` 三种角色，分别对应系统、机器人和用户，其中，system可以理解为我们对当前上下午设定的机器人角色背景。

In [12]:
engine = "gpt-35-turbo"

response = openai.ChatCompletion.create(engine=engine, 
                                        messages=[
                                            {"role":"system", "content":"你是一个有用的助理"},
                                            {"role":"user", "content":"你是谁?"},
                                        ],
                                        temperature=0.1,
                                        max_tokens=1024
                                        )

In [13]:
print(response["choices"][0]["message"]["content"])
response

我是一个AI语言模型，可以回答你的问题和提供帮助。


<OpenAIObject chat.completion id=chatcmpl-82shHJDEWOKBX40qyIawGbcLYPhm6 at 0x107b1ac70> JSON: {
  "id": "chatcmpl-82shHJDEWOKBX40qyIawGbcLYPhm6",
  "object": "chat.completion",
  "created": 1695697603,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "\u6211\u662f\u4e00\u4e2aAI\u8bed\u8a00\u6a21\u578b\uff0c\u53ef\u4ee5\u56de\u7b54\u4f60\u7684\u95ee\u9898\u548c\u63d0\u4f9b\u5e2e\u52a9\u3002"
      }
    }
  ],
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 27,
    "total_tokens": 51
  }
}

# 让GPT记住前面的聊天记录

In [14]:
def create_msg(prompt:str, is_user:bool=True) -> dict:
    role = "user" if is_user else "assistant"
    return {"role":role, "content":prompt}
    

## 没有记忆的

In [15]:
def chatCompletionInstant(prompt, temperature=0.1, max_tokens=1024, n=1) -> (str, dict):
    msg = [{"role": "system", "content": "you are a helpful assistant"}, create_msg(prompt)]
    
    response = openai.ChatCompletion.create(engine=engine, 
                                            messages=msg,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            n=n)
    return response["choices"][0]["message"]["content"], response

In [16]:
res, _ = chatCompletionInstant("20个字简单说说什么是光刻机")
print(res)
res, _ = chatCompletionInstant("我前面说了什么")
print(res)

光刻机是一种半导体制造设备，用于在芯片表面上制造微小的图案和结构。
抱歉，我无法回答这个问题，因为我无法访问您的之前的对话记录。


## 有记忆的

In [18]:
messages = [{"role":"system", "content":"you are a helpful assistant"}]
def chatComplete(prompt, temperature=0.1, max_tokens=1024, top_p=1, n=1):
    messages.append(create_msg(prompt))
    response = openai.ChatCompletion.create(engine=engine,
                                            messages=messages,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            n=n,
                                            top_p=top_p
                                            )
    return response["choices"][0]["message"]["content"], response


In [19]:
res, _ = chatComplete("20个字简单说说什么是光刻机")
print(res)
res, _ = chatComplete("我前面说了什么")
print(res)

光刻机是一种半导体制造设备，用于在芯片表面上制造微小的图案和结构。
您前面说了"20个字简单说说什么是光刻机"。


In [20]:
messages

[{'role': 'system', 'content': 'you are a helpful assistant'},
 {'role': 'user', 'content': '20个字简单说说什么是光刻机'},
 {'role': 'user', 'content': '我前面说了什么'}]

# Token 计算

In [21]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0613")

# 去掉 Key, 只保留 Value
messages=[
    {"system", "You are a helpful assistant."},
    {"user",  "Can you tell me who are you?"},
]

print(f"Prompt_tokens: {len(encoding.encode(str(messages)))}")

Prompt_tokens: 25


In [28]:
messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user", "content":"Can you tell me who are you?"},
        ]
msg = [set(d.values()) for d in messages]
print(f"Prompt_tokens: {len(encoding.encode(str(msg)))}")

Prompt_tokens: 25


In [29]:
msg==[
    {"system", "You are a helpful assistant."},
    {"user",  "Can you tell me who are you?"},
]

True

In [30]:
msg

[{'You are a helpful assistant.', 'system'},
 {'Can you tell me who are you?', 'user'}]